In [87]:
from google.colab import files

uploaded = files.upload()
video_files = list(uploaded.keys())
print(f"Uploaded videos: {video_files}")


Saving vid (1).mp4 to vid (1) (6).mp4
Saving vid (2).mp4 to vid (2) (4).mp4
Saving vid (3).mp4 to vid (3) (5).mp4
Saving vid (4).mp4 to vid (4) (5).mp4
Saving vid (5).mp4 to vid (5) (4).mp4
Saving vid (6).mp4 to vid (6) (4).mp4
Saving vid (7).mp4 to vid (7) (4).mp4
Uploaded videos: ['vid (1) (6).mp4', 'vid (2) (4).mp4', 'vid (3) (5).mp4', 'vid (4) (5).mp4', 'vid (5) (4).mp4', 'vid (6) (4).mp4', 'vid (7) (4).mp4']


In [88]:
import cv2
import cvzone
from cvzone.ColorModule import ColorFinder
import numpy as np
import math

# Color detection settings
hsvVals = {'hmin': 8, 'smin': 124, 'vmin': 13, 'hmax': 24, 'smax': 255, 'vmax': 255}

# Function to process a single video and return processed frames list
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    myColorFinder = ColorFinder(False)

    posListX = []
    posListY = []
    listX = list(range(0, 1300))

    start = True
    prediction = False
    output_frames = []

    while True:
        success, img = cap.read()
        if not success:
            print(f"End of video or cannot read frame: {video_path}")
            break

        h, w = img.shape[:2]
        crop_h = min(900, h)
        img = img[0:crop_h, :]

        imgPrediction = img.copy()
        imgResult = img.copy()

        imgBall, mask = myColorFinder.update(img, hsvVals)
        imgCon, contours = cvzone.findContours(img, mask, minArea=200)

        if contours:
            posListX.append(contours[0]['center'][0])
            posListY.append(contours[0]['center'][1])

        if posListX:
            if len(posListX) < 18:
                coff = np.polyfit(posListX, posListY, 2)

            for i, (posX, posY) in enumerate(zip(posListX, posListY)):
                pos = (posX, posY)
                cv2.circle(imgCon, pos, 10, (0, 255, 0), cv2.FILLED)
                cv2.circle(imgResult, pos, 10, (0, 255, 0), cv2.FILLED)

                if i == 0:
                    cv2.line(imgCon, pos, pos, (0, 255, 0), 2)
                    cv2.line(imgResult, pos, pos, (0, 255, 0), 2)
                else:
                    cv2.line(imgCon, (posListX[i - 1], posListY[i - 1]), pos, (0, 255, 0), 2)
                    cv2.line(imgResult, (posListX[i - 1], posListY[i - 1]), pos, (0, 255, 0), 2)

            for x in listX:
                y = int(coff[0] * x ** 2 + coff[1] * x + coff[2])
                cv2.circle(imgPrediction, (x, y), 2, (255, 0, 255), cv2.FILLED)
                cv2.circle(imgResult, (x, y), 2, (255, 0, 255), cv2.FILLED)

            # Prediction logic
            if len(posListX) < 10:
                a, b, c = coff
                c = c - 593
                disc = b ** 2 - 4 * a * c
                if disc >= 0:
                    x_intercept = int((-b - math.sqrt(disc)) / (2 * a))
                    prediction = 300 < x_intercept < 430
                else:
                    prediction = False

            if prediction:
                cvzone.putTextRect(imgResult, "Basket", (50, 150), colorR=(0, 200, 0),
                                   scale=5, thickness=10, offset=20)
            else:
                cvzone.putTextRect(imgResult, "No Basket", (50, 150), colorR=(0, 0, 200),
                                   scale=5, thickness=10, offset=20)

        cv2.line(imgCon, (330, 593), (430, 593), (255, 0, 255), 10)
        imgResult = cv2.resize(imgResult, (0, 0), None, 0.7, 0.7)

        output_frames.append(imgResult)

    cap.release()
    return output_frames


In [89]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_paths = []

for idx, video_file in enumerate(video_files, start=1):
    print(f"Processing video {idx}: {video_file}")
    frames = process_video(video_file)

    if len(frames) == 0:
        print(f"No frames found for {video_file}. Skipping saving.")
        continue

    height, width, _ = frames[0].shape
    out_path = f'output_video_{idx}.mp4'
    out = cv2.VideoWriter(out_path, fourcc, 20, (width, height))

    for frame in frames:
        out.write(frame)
    out.release()

    output_paths.append(out_path)
    print(f"Saved output video: {out_path}")


Processing video 1: vid (1) (6).mp4


<ipython-input-88-ef66fc1c0cbc>:45: RankWarning: Polyfit may be poorly conditioned
  coff = np.polyfit(posListX, posListY, 2)
<ipython-input-88-ef66fc1c0cbc>:45: RankWarning: Polyfit may be poorly conditioned
  coff = np.polyfit(posListX, posListY, 2)


End of video or cannot read frame: vid (1) (6).mp4
Saved output video: output_video_1.mp4
Processing video 2: vid (2) (4).mp4


<ipython-input-88-ef66fc1c0cbc>:45: RankWarning: Polyfit may be poorly conditioned
  coff = np.polyfit(posListX, posListY, 2)
<ipython-input-88-ef66fc1c0cbc>:45: RankWarning: Polyfit may be poorly conditioned
  coff = np.polyfit(posListX, posListY, 2)


End of video or cannot read frame: vid (2) (4).mp4
Saved output video: output_video_2.mp4
Processing video 3: vid (3) (5).mp4


...

End of video or cannot read frame: vid (5) (4).mp4
Saved output video: output_video_5.mp4
Processing video 6: vid (6) (4).mp4


<ipython-input-88-ef66fc1c0cbc>:45: RankWarning: Polyfit may be poorly conditioned
  coff = np.polyfit(posListX, posListY, 2)
<ipython-input-88-ef66fc1c0cbc>:45: RankWarning: Polyfit may be poorly conditioned
  coff = np.polyfit(posListX, posListY, 2)


End of video or cannot read frame: vid (6) (4).mp4
Saved output video: output_video_6.mp4
Processing video 7: vid (7) (4).mp4


<ipython-input-88-ef66fc1c0cbc>:45: RankWarning: Polyfit may be poorly conditioned
  coff = np.polyfit(posListX, posListY, 2)
<ipython-input-88-ef66fc1c0cbc>:45: RankWarning: Polyfit may be poorly conditioned
  coff = np.polyfit(posListX, posListY, 2)


End of video or cannot read frame: vid (7) (4).mp4
Saved output video: output_video_7.mp4


In [90]:
from google.colab import files

for out_path in output_paths:
    print(f"Downloading {out_path}...")
    files.download(out_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [93]:
from IPython.display import HTML
from base64 import b64encode

# Choose the video to play (e.g., first one)
video_path = output_paths[0]  # Change index 0–6 to play other videos

# Read and encode video for HTML5 playback
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

# Embed video in HTML player
HTML(f"""
<video width=700 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")
